In [ ]:
import os

from ipyleaflet import Map, SearchControl
center = [19.1646, 72.8493]
m = Map(center=center, zoom=3)

In [ ]:
search = SearchControl(position="topleft", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=10,
                       propertyName='display_name')
m.add_control(search)
m